#### Import Fablib ####

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
import time
fablib = fablib_manager()

#### Add sink/monitor to slice ####

In [ ]:
slice_name = 'TrafficSilce'

site1 = 'WASH'
site2 = 'GPN'

#names for sink/monitor nodes 
target_name = 'sink'
monitor_name = 'monitor' 

#networks for sink/monitor 
snet_name='sink_net'
mnet_name='monitor_net'

#nics for sink/monitor 
sink_nic = 's_nic'
monitor_nic = 'm_nic'

In [ ]:
try:
    #Create slice 
    slice = fablib.new_slice(name=slice_name)
    # sink
    sink = slice.add_node(name=target_name, site=site1)
    s_iface  = sink.add_component(model='NIC_Basic', name=sink_nic).get_interfaces()[0]
    s_net = slice.add_l3network(name=snet_name, interfaces=[s_iface], type='IPv6')
    
    #monitor 
    monitor = slice.add_node(name=monitor_name, site=site2)
    m_iface = monitor.add_component(model='NIC_Basic',name=monitor_nic).get_interfaces()[0]
    m_net = slice.add_l3network(name=mnet_name, interfaces=[m_iface], type='IPv6')
    
  
except Exception as e:
    print(f"Exception: {e}")

#### Add generators ####

In [ ]:
#set the number of generators for the experiment
num_generators = 2
#array of sites for generators to run on - 2 in this example
g_sites = ["TACC","DALL"]
# could also pass in an array here if code adjusted slightly  
g_model = 'NIC_Basic'

#ubuntu not working fablib 1.3.8
#g_image ='default_ubuntu_20' 

#get/contain references- reference lost after slice provisioned 
generators = []
g_ifaces = []
g_networks = [] 

# used to get reference from slice 
generator_names = []

generator_iface_names = []

generator_network_names =[]
 
generators_ips = []

##### auto generate names #####

In [ ]:
for i in range(num_generators):
    # variables for node  
    generator_names.append("generator"+ str(i))
    generator_iface_names.append("giface" + str(i))
    generator_network_names.append("gnetwork" + str(i))

In [ ]:
try:
    for i in range(num_generators):        
        # node, interface, network to slice 
        generators.append(slice.add_node(name = generator_names[i],site = g_sites[i]))
        
        g_ifaces.append(generators[i].add_component(model=g_model,name=generator_iface_names[i]).get_interfaces()[0])
        
        g_networks.append(slice.add_l3network(name=generator_network_names[i],interfaces=[g_ifaces[i]],type='IPv6'))
        
except Exception as e:
    print(f"Exception: {e}")

#### submit the slice ####

In [ ]:
slice.submit()

#### config for sink/monitor ####

##### sink #####

In [ ]:
try:
    s_net = slice.get_network(name=snet_name)
    sink_addr = s_net.get_available_ips().pop(0)
    sink = slice.get_node(name=target_name)
    s_iface = sink.get_interface(network_name=snet_name)
    s_iface.ip_addr_add(addr=sink_addr,subnet=s_net.get_subnet())
    sink.ip_route_add(subnet=slice.get_network(name=mnet_name).get_subnet(),gateway=s_net.get_gateway())
    
except Exception as e:
    print(f"Exception: {e}")

<h5> monitor </h5>

In [ ]:
try:
    m_net = slice.get_network(name=mnet_name)
    monitor_addr = m_net.get_available_ips().pop(0)
    monitor = slice.get_node(name=monitor_name)
    m_iface = monitor.get_interface(network_name=mnet_name)
    m_iface.ip_addr_add(addr=monitor_addr,subnet=m_net.get_subnet())
    monitor.ip_route_add(subnet=s_net.get_subnet(),gateway=m_net.get_gateway())
except Exception as e:
    print(f"Exception: {e}")

<h4> config for generators </h4>

In [ ]:
try:
    for i in range(num_generators): 
        cnet = slice.get_network(name=generator_network_names[i])
        ips = cnet.get_available_ips()
        
        cur = slice.get_node(name=generator_names[i])
        ciface = cur.get_interface(network_name=generator_network_names[i])
          
        cip = ips.pop(0)
        generators_ips.append(cip)
        
        ciface.ip_addr_add(addr=cip,subnet=cnet.get_subnet())
        
        cur.ip_route_add(subnet=s_net.get_subnet(),gateway=cnet.get_gateway())
        sink.ip_route_add(subnet=cnet.get_subnet(), gateway=s_net.get_gateway())
    
        
except Exception as e:
    print(f"Exception: {e}")

#### install tools for experiment ####

In [ ]:
combo = "sudo yum install iperf3 -y"

In [ ]:
sink = slice.get_node(name='sink')
sink.execute(combo)

In [ ]:
for i in range(num_generators):
    generator = slice.get_node(name=generator_names[i])
    generator.execute(combo)

#### configure experiment ####

In [ ]:
start_port = 5201
num_processes = 50

monitor = slice.get_node(name="monitor")
sink = slice.get_node(name="sink")
monitor_com = "ping -c 500 " + str(sink_addr)


#### control ####

In [ ]:
try:
    threads = {}
    threads[monitor] = monitor.execute_thread(monitor_com)
    time.sleep(10)
    
    res,stderr = threads[monitor].result()
    
except Exception as e:
    print(f"Exception: {e}")

#### experimental ####

In [ ]:
try:
    threads = {}
    
    #monitor thread pings the sink (target) contionusly 
    threads[monitor] = monitor.execute_thread(monitor_com)
    time.sleep(10)
    
    #ports start on 5201 and increase based on number of processes 
    port = start_port
    #start specified number of iperf3 processses on each of the generator nodes. 
    for i in range(num_processes):
        for j in range(num_generators):
            #command to be executed on the sink node
            server_com = "iperf3 -s -p " + str(port) + "&"
            #command to be executed on the generator node
            client_com = "iperf3 -c " + str(sink_addr) + " -n 10G -l 10240B -p " + str(port) + "&"

            cur_att = slice.get_node(name=generator_names[j])
            #execute command on sink thread
            threads[sink] = sink.execute_thread(server_com)
            time.sleep(.75)
            #execute command on current generator node 
            threads[generators[j]] = cur_att.execute_thread(client_com)
            port += 1 
            
        
    #wait for ping results (500 pings) 
    stdout,stderr = threads[monitor].result()
    
except Exception as e:
    print(f"Exception: {e}")

#### process data ####

In [ ]:
results = []
results.append(res)
results.append(stdout)

In [ ]:
import re
import matplotlib.pyplot as plt
curves = []
for r in results:
    data = []

    for line in re.findall("time=.*",r):
        datapoint = re.findall('\d\.\d+',line.rstrip())
        toAdd = datapoint[0]
        if isinstance(toAdd,str):
            data.append(float(toAdd))
    curves.append(data)
curves[0].pop(0)
for c in curves:
    plt.plot(c)
    plt.ylim(ymin=0)
    plt.ylabel('ping times(ms)')
    plt.xlabel('sequence')
    plt.show()
for c in curves:
    plt.plot(c)
    plt.ylim(ymin=0)
    plt.ylabel('ping times(ms)')
    plt.xlabel('sequence')
plt.show()